In [1]:
from pathlib import Path
import os
import kaggle
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
import optuna

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
else:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)
    if not orig_path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

SEED = 42
SPLITS = 5
VERSION = 6
OUTPUT = f'extra_tree_clf_submisson_v{VERSION}.csv'

np.random.seed(SEED)

/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv(path/'train.csv', index_col = 'id')
test = pd.read_csv(path/'test.csv', index_col = 'id')
orig_train = pd.read_csv(orig_path/'jm1.csv')

In [3]:
for object_features in list(orig_train.loc[:, orig_train.dtypes == 'O']):
    orig_train[object_features] = orig_train[object_features].replace({'?' : np.nan}).astype('float64')

In [4]:
X = pd.concat([train, orig_train])
y = X.pop('defects')

In [5]:
partial_pipeline = make_pipeline(ColumnTransformer([('drop', 'drop',
                                  ['iv(g)', 't', 'b', 'n',
                                   'lOCode', 'v', 'branchCount',
                                   'e', 'i', 'lOComment'])],
                                remainder='passthrough'), SimpleImputer(), FunctionTransformer(func=np.log1p, validate=False), PowerTransformer())

In [6]:
from sklearn.model_selection import cross_val_score

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 600, 100),
        'n_jobs': trial.suggest_categorical('n_jobs', [-1]),
        'random_state': trial.suggest_categorical('random_state', [SEED]),
        'max_depth': trial.suggest_int('max_depth', 4, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 150),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 60),
        'criterion': trial.suggest_categorical("criterion", ["gini"]),
        'max_features': trial.suggest_categorical('max_features', [1.0]),
    }

    pipeline = make_pipeline(partial_pipeline, ExtraTreesClassifier(**params))
    scores = cross_val_score(pipeline, X, y, cv=SPLITS, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30, show_progress_bar=True)

[I 2023-10-17 15:25:48,290] A new study created in memory with name: no-name-bb62fa7d-71db-4cac-961c-ec22fc4dd7d3
Best trial: 0. Best value: 0.783927:   3%|█████▍                                                                                                                                                              | 1/30 [01:26<41:40, 86.21s/it]

[I 2023-10-17 15:27:14,500] Trial 0 finished with value: 0.7839273647200006 and parameters: {'n_estimators': 500, 'n_jobs': -1, 'random_state': 42, 'max_depth': 19, 'min_samples_split': 69, 'min_samples_leaf': 18, 'criterion': 'gini', 'max_features': 1.0}. Best is trial 0 with value: 0.7839273647200006.


Best trial: 1. Best value: 0.784094:   7%|██████████▉                                                                                                                                                         | 2/30 [02:39<36:39, 78.55s/it]

[I 2023-10-17 15:28:27,686] Trial 1 finished with value: 0.7840938716669501 and parameters: {'n_estimators': 400, 'n_jobs': -1, 'random_state': 42, 'max_depth': 47, 'min_samples_split': 43, 'min_samples_leaf': 34, 'criterion': 'gini', 'max_features': 1.0}. Best is trial 1 with value: 0.7840938716669501.


Best trial: 2. Best value: 0.784205:  10%|████████████████▍                                                                                                                                                   | 3/30 [03:35<30:39, 68.14s/it]

...

[I 2023-10-17 15:54:53,846] Trial 27 finished with value: 0.7845802907578572 and parameters: {'n_estimators': 600, 'n_jobs': -1, 'random_state': 42, 'max_depth': 26, 'min_samples_split': 50, 'min_samples_leaf': 42, 'criterion': 'gini', 'max_features': 1.0}. Best is trial 15 with value: 0.785717273334426.


Best trial: 15. Best value: 0.785717:  97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌     | 29/30 [29:26<00:59, 59.32s/it]

[I 2023-10-17 15:55:14,451] Trial 28 finished with value: 0.7845210716755463 and parameters: {'n_estimators': 200, 'n_jobs': -1, 'random_state': 42, 'max_depth': 8, 'min_samples_split': 92, 'min_samples_leaf': 51, 'criterion': 'gini', 'max_features': 1.0}. Best is trial 15 with value: 0.785717273334426.


Best trial: 15. Best value: 0.785717: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [30:47<00:00, 61.58s/it]

[I 2023-10-17 15:56:35,584] Trial 29 finished with value: 0.7841462613035993 and parameters: {'n_estimators': 500, 'n_jobs': -1, 'random_state': 42, 'max_depth': 20, 'min_samples_split': 67, 'min_samples_leaf': 28, 'criterion': 'gini', 'max_features': 1.0}. Best is trial 15 with value: 0.785717273334426.


In [7]:
study.best_params, study.best_value

({'n_estimators': 500,
  'n_jobs': -1,
  'random_state': 42,
  'max_depth': 12,
  'min_samples_split': 101,
  'min_samples_leaf': 47,
  'criterion': 'gini',
  'max_features': 1.0},
 0.785717273334426)

In [8]:
pipeline = make_pipeline(partial_pipeline, ExtraTreesClassifier(**study.best_params))
pipeline.fit(X, y)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('drop',
                                                                   'drop',
                                                                   ['iv(g)',
                                                                    't', 'b',
                                                                    'n',
                                                                    'lOCode',
                                                                    'v',
                                                                    'branchCount',
                                                                    'e', 'i',
                                                                    'lOComment'])])),
                                 ('simpleimputer', SimpleImputer()),
                                 ('functiontransformer',
                                  FunctionTransformer(func=<ufunc 'log1p'>)),
                                 ('powertransformer', PowerTransformer())])),
                ('extratreesclassifier',
                 ExtraTreesClassifier(max_depth=12, max_features=1.0,
                                      min_samples_leaf=47,
                                      min_samples_split=101, n_estimators=500,
                                      n_jobs=-1, random_state=42))])

In [9]:
submission = test.copy()
submission['defects'] = pipeline.predict_proba(submission)[:, 1]
submission.defects.to_csv(OUTPUT)